<!-- -*- mode: markdown; coding: utf-8; fill-column: 60; ispell-dictionary: "english" -*- -->

<meta charset="utf-8"/>
<meta name="viewport" content="width=device-width,initial-scale=1"/>
<link rel="stylesheet" href="style.css">


# Lab 2 - Testing the database

As usual we have to tell Jupyter to allow SQL:

In [7]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


And then we import our movie database

In [8]:
%sql sqlite:///movies.db

'Connected: @movies.db'

We want to make sure that SQLite3 really checks our foreign
key constraints -- to do that, we run:

In [9]:
%%sql
PRAGMA foreign_keys=ON;

 * sqlite:///movies.db
Done.


[]

Now write SQL code for the following tasks:


+ Show the names of all movies.

In [10]:
%%sql
SELECT title FROM movies;

 * sqlite:///movies.db
Done.


title
Inception
Spotlight


+ Show the performance dates for one of the movies.

In [16]:
%%sql
SELECT start_date
FROM performances
WHERE imdb='034256';

 * sqlite:///movies.db
Done.


start_date
2019–02-01


+ Show all data concerning performances at a given theatere
  on a given date.

In [27]:
%%sql
SELECT * 
FROM performances 
WHERE t_name = 'Filmstaden'
AND start_date = '2019-02-01';

 * sqlite:///movies.db
Done.


p_id,start_time,start_date,imdb,t_name
c972ad3788214c530e95a88680a8d62b,17:00,2019-02-01,034256,Filmstaden


+ List all customers

In [28]:
%%sql
SELECT c_name FROM customers;

 * sqlite:///movies.db
Done.


c_name
Marcus_Hedebark
Olle_Römer


+ List all tickets

In [30]:
%%sql
SELECT * FROM tickets;

 * sqlite:///movies.db
Done.


ticket_id,p_id,user_name
a6755734b7fda532aac4a0ec108f4e3a,c972ad3788214c530e95a88680a8d62b,movieMan13


+ Create a new ticket to some performance (i.e., insert a
  new row in your table of tickets).

In [40]:
%%sql
INSERT
INTO tickets (p_id, user_name)
SELECT p_id, 'iLoveMovies'
FROM performances
LIMIT 1

 * sqlite:///movies.db
Done.


[]

In
  [PostgreSQL](https://www.postgresql.org/docs/current/sql-insert.html)
  we can get any value generated during an insert using the
  `INSERT...-RETURNING` statement:

~~~{.sql}
INSERT
INTO       students
VALUES     ('Amy', 3.9, 1200)
RETURNING  s_id
~~~


In SQLite3 (as of January 2019), we can't do that, instead
  we can the following idea: each row in a SQLite3 table has
  a `rowid` attribute, it is a unique integer which
  essentially tells in which order the rows were inserted,
  and it's not displayed in queries, unless we ask for it.
  SQLite3 also have a function, `last_insert_rowid()`, which
  returns the `rowid` of the last inserted row of a table,
  so we can see the `s_id` of the most recently inserted
  student with the following query:

~~~{.sql}
SELECT s_id
FROM   students
WHERE  rowid = last_insert_rowid();
~~~


Now, check what ticket number we got for the ticket we
  created above (it should be the same as the ticket id,
  which should be a `randomblob`):

In [41]:
%%sql
SELECT ticket_id
FROM tickets
WHERE rowid = last_insert_rowid();

 * sqlite:///movies.db
Done.


ticket_id
d61b614326c0e5f204f209416d6fbdbf


+ Try to insert two movie theaters with the same name (this
  should fail).

In [59]:
%%sql
INSERT INTO theatres (t_name, cap)
VALUES ('Bio', 200),
       ('Bio', 300);

 * sqlite:///movies.db
1 rows affected.


[]

+ Try to insert a performance where the theater doesnâ€™t
  exist in the database (this should fail).

In [54]:
%%sql
INSERT INTO performances (start_time, start_date, imdb, t_name)
VALUES ('17:00', '2019-02-14', 'asdasd123123 ', 'Theatre3');


 * sqlite:///movies.db


IntegrityError: (sqlite3.IntegrityError) FOREIGN KEY constraint failed [SQL: "INSERT INTO performances (start_time, start_date, imdb, t_name)\nVALUES ('17:00', '2019-02-14', 'asdasd123123 ', 'Theatre3');"] (Background on this error at: http://sqlalche.me/e/gkpj)

+ Create a ticket where either the user or the performance
  doesnâ€™t exist.

In [61]:
%%sql
INSERT INTO tickets (user_name)
VALUES ('movieMan13');


 * sqlite:///movies.db
1 rows affected.


[]

In [62]:
%%sql
SELECT *
FROM tickets

 * sqlite:///movies.db
Done.


ticket_id,p_id,user_name
a6755734b7fda532aac4a0ec108f4e3a,c972ad3788214c530e95a88680a8d62b,movieMan13
03f859aade5430db07416c2bd136ffc5,2cef44134bf5429cca281dc1e7572ee9,iLoveMovies
71cf0bf7d70661d3d6d50e5a469ed812,2cef44134bf5429cca281dc1e7572ee9,iLoveMovies
da19dc5d96167af44e76630c457b9891,4f8889dfb55c7bbf5d90d161ddc1ae87,iLoveMovies
d61b614326c0e5f204f209416d6fbdbf,2cef44134bf5429cca281dc1e7572ee9,iLoveMovies
b018a2bba646c2c1723ec62a9ffae914,None,movieMan13
